# From SQL to pandas challenge 8

In [1]:
# import libraries
import pandas as pd

# load data
# This code is made to load our data stored on Google Drive
def gd_path(file_id):
    """Generate a shareable link from Google Drive file id."""
    return f"https://drive.google.com/uc?export=download&id={file_id}"

# Google Drive file ids
files_id = {
    "titleauthor": "1F1JOiYXStWacOBca6coNVfyVtoST7ZgD",
    "titles": "1PLdn50N9GRa53ZbuVWo0l47F_IXdvlEm",
    "stores": "1f-GCgip7O93CpbAkYvOsc21eKnSOSHsQ", 
    "sales": "1fzFc9rwYmVIPaGOFmhLVxCi3kg19vNU2", 
    "roysched": "1zPRZPoFPEMKyrNR5VSENeYFHGCBZmxbs", 
    "publishers": "1s9E8_AVOziTrowb3wyh2jg3PV763VOyq",
    "pub_info": "1OEgogcGKy--EpuVj0kqq7lyBZNGW6YSv", 
    "jobs": "1V1Za8hUdXD-vJOyRdX4aQV5wanIff2eM", 
    "employee": "1h9mUjsVqpP74b1w0x7KOw37n_n9Ulkt5", 
    "discounts": "111dvSxMcCsTgOuV1wDSKFJxO1Xcxd9VS", 
    "authors": "1fEF89Nhe61EebAljKlwFwfEuokK0o6aJ"
}

# Read data from Google Drive
sales = pd.read_csv(gd_path(files_id["sales"]), sep=";")
authors = pd.read_csv(gd_path(files_id["authors"]), sep=";")

## 1. Select the top 5 sales with most quantity sold between 1993-03-11 and 1994-09-13 from the table sales

In [2]:
sales['ord_date'] = pd.to_datetime(sales['ord_date']) 
(
    sales[(sales['ord_date'] >= '1993-03-11') & (sales['ord_date'] <= '1994-09-13')]
    .sort_values(by = 'qty', ascending = False).head(5)
)

,stor_id,ord_num,ord_date,qty,payterms,title_id
3,7066,QA7442.3,1994-09-13,75,ON invoice,PS2091
2,7066,A2976,1993-05-24,50,Net 30,PC8888
20,8042,QA879.1,1993-05-22,30,Net 30,PC1035
11,7131,P3087a,1993-05-29,25,Net 60,PS2106
13,7131,P3087a,1993-05-29,25,Net 60,PS7777


### Hint:

In SQL the syntax is:

```sql
SELECT *
FROM sales
WHERE ord_date BETWEEN '1993-03-11' AND '1994-09-13'
ORDER BY qty DESC
LIMIT 5;
```

## 2. How many authors have an "i" in their first name, are from Utah, Maryland, or Kansas?

In [5]:
(
authors.query('state in ("CA" , "MD", "KS")')[['au_fname','state']]
  .loc[authors["au_fname"].str.contains("i", case = False, regex = True)][["au_fname"]].count()
)

au_fname    6
dtype: int64

### Hint:

In SQL the syntax is:

```sql
SELECT COUNT(*)
FROM authors
WHERE au_fname LIKE '%i%'
AND state IN ('CA' , 'MD', 'KS');
```

## 3. In California, how many authors are there in cities that contain an "o" in the name?
   - Show only results for cities with more than 1 author.
   - Sort the cities ascendingly by author count.

In [6]:
authors[(authors.state=='CA')].groupby('city')['au_id'].count().loc[lambda x : x > 1]

city
Berkeley     2
Oakland      5
Palo Alto    2
Name: au_id, dtype: int64

In [7]:
authors_ca = authors.query('state in ("CA")')[['city','au_fname']]
authors_ca_o = authors_ca.loc[authors["city"].str.contains("o", case = False)]
authors_ca_o_count = authors_ca_o.groupby(by=["city"]).agg(count=('city', 'size'))
authors_ca_o_count[(authors_ca_o_count['count'] >= 2)]

,count
city,
Oakland,5
Palo Alto,2


### Hint:

In SQL the syntax is:

```sql
SELECT city, COUNT(au_fname)
FROM authors
WHERE state = 'CA' 
AND city LIKE '%o%'
GROUP BY city
HAVING COUNT(au_fname) > 1
ORDER BY COUNT(au_fname);
```